In [ ]:
import cv2
import numpy as np
import socket
import time
import threading

In [ ]:
def send_d():
    s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    s.connect(("127.0.0.1",2345))
    s.send(b"connection established , port_no = 2345")
    cap = cv2.VideoCapture(0)
    print("Live streaming started ")
    while True:
        capture,photo= cap.read()
        if capture:
            photo = cv2.resize(photo,(600,500))
            s.send(np.ndarray.tobytes(photo))
        else:
            print("No image to send")
        
def recv_d():
    r = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    r.bind(("127.0.0.1",1234))
    r.listen()
    sess,add = r.accept()
    print(sess.recv(1024))
    print("Live streaming started ")
    while True:
        data_r = sess.recv(100000000)
        image_r = np.frombuffer(data_r , np.uint8)
        if len(image_r) == 600*500*3:
            cv2.imshow("live streaming",image_r.reshape(500,600,3))
            if cv2.waitKey(10) == 13:
                cv2.destroyAllWindows()
                break
        else:
            print("frame lost")

send_t = threading.Thread(target = send_d).start()
recv_t = threading.Thread(target = recv_d).start()
